In [2]:
library(fit)
library(tidyverse)
#gpxsee to inspect fit files

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



# Decoder script
## Helper functions

In [3]:
#convert timestamp to date (weird, but off by only some minutes)
to_date <- function (t) {as.POSIXct(as.numeric(as.character(t)), origin="1990-01-01", tz="GMT+23")}
#array: hours k[1], mins k[2], secs k[3] (from int (seconds))
to_duration <- function (t,x="h") {
            if(x!="s"){
                t <- round(t/1000) #normalize if t comes from f.ex total_elapsed_time
            }
            hours <- t%/%3600
            mins <- t%%3600%/%60
            secs <- t%%60
            paste(hours,":",floor(mins),":",floor(secs))
             }
#km/h -> min/km, mins k[1], secs k[2]
to_pace <- function (t){
    h <- 60/t
    mins <- floor(h)
    secs <- (h-mins)*60
    if(mins > 60){
        mins <- "NaN"
    }
    paste(mins,":",floor(secs))
}
#m/s -> km/h
to_kmh <- function (ms) {ms*3.6}
#m/s -> min/100m 
to_swimpace <- function (ms) {10**4/(ms*6)}

In [4]:
#input: int from FIT, out: string
get_sport <- function(f){
    case_when(
        f == 2 ~ "Bike",
        f == 17 ~ "Hiking",
        f == 1 ~ "Running",
        f == 5 ~ "Swimming",
        TRUE ~ "UNKNOWN_SPORT"
    )
}

In [5]:
get_running_session <- function(buf){
    data.frame("avg_hr"            = buf$avg_heart_rate, 
               "max_hr"            = buf$max_heart_rate,
               "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
               "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
               "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
               "max_pace"          = to_pace(round(to_kmh(buf$max_speed/1000),1)),
               "total_ascent"      = buf$total_ascent, # TODO 
               "total_descent"     = buf$total_descent, # TODO
               "total_distance"    = round(buf$total_distance/10**5,2),
               "total_time"        = to_duration(buf$total_elapsed_time),
               "total_time_moving" = to_duration(buf$total_timer_time),
               "num_laps"          = buf$num_laps,
               "training_effect"   = buf$total_training_effect/10
              )
}
get_running_lap <- function(buf){
    data.frame("avg_hr"            = buf$avg_heart_rate,
               "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
               "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
               "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
               #"time"            = to_date(buf$timestamp),
               "total_distance"    = round(buf$total_distance/10**5,2),
               "total_time"        = to_duration(buf$total_elapsed_time),
               "total_time_moving" = to_duration(buf$total_timer_time)
              )
}

## Start decoding here!

Plan: Read all files in ACTIVITY directory, and convert all *.FIT files to *.JSON files.

The *.FIT files are deleted then

In [9]:
act_files <- list.files('f/')
#contains all fit files to be processed
fit_files <- c()
for (f in act_files){
    sp <- unlist(strsplit(f,"[.]"))
    #print(sp[1:length(sp)-1])
    if(sp[length(sp)] == "FIT" | sp[length(sp)] == "fit"){
        #file is a fit file! save its name
        fit_files <- c(fit_files,sp[1:length(sp)-1])
    }
}

In [10]:
file <- read.fit(paste('f/',fit_files[1],'.FIT',sep=""))

In [13]:
file$record

accumulated_power,altitude,cadence,distance,fractional_cadence,heart_rate,position_lat,position_long,speed,timestamp
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
65535,4253,0,0,0,120,48.52734,9.199440,0,953220649
65535,4253,0,0,0,120,48.52733,9.199429,0,953220650
65535,4253,58,1091,64,116,48.52736,9.199243,2137,953220658
65535,4253,84,2522,0,109,48.52736,9.199049,3387,953220662
65535,4252,84,2925,0,103,48.52736,9.198994,3798,953220663
65535,4252,86,4479,0,99,48.52735,9.198785,3919,953220667
65535,4252,86,4862,0,104,48.52735,9.198733,3863,953220668
65535,4252,85,5228,0,108,48.52735,9.198684,3863,953220669
65535,4252,85,5598,0,115,48.52735,9.198634,3863,953220670


In [11]:
file_sport_type <- get_sport(file$sport$sport)
file_date <- to_date(file$session$start_time)

#for running
file_session <- get_running_session(file$session[,c('avg_heart_rate','max_heart_rate',
                                                    'avg_speed','max_speed',
                                                    'total_ascent','total_descent', 'total_distance',
                                                    'total_elapsed_time', 'total_timer_time',
                                                    'num_laps', 'total_training_effect')])
file_lap <- get_running_lap(file$lap[,c('avg_heart_rate','avg_speed','max_speed','timestamp',
                                        'total_distance','total_elapsed_time','total_timer_time')])
print(file$record[1,])
buf <- file$record[,c('distance','heart_rate','speed','position_lat','position_long','timestamp')]
head(buf)
data.frame("distance" = round(buf$distance/10**5,3),
           "hr" = buf$heart_rate,
           "speed" = round(to_kmh(buf$speed/1000),1),
           "pace" = to_pace(round(to_kmh(buf$speed/1000),1)),
           "pos_lat" = buf$position_lat,
           "pos_long" = buf$position_long,
           "time" = to_duration(buf$timestamp-file$session$start_time,"s"))

Warning message in if (mins > 60) {:
“Bedingung hat Länge > 1 und nur das erste Element wird benutzt”


  accumulated_power altitude cadence distance fractional_cadence heart_rate
1             65535     4253       0        0                  0        120
  position_lat position_long speed timestamp
1     48.52734       9.19944     0 953220649


,distance,heart_rate,speed,position_lat,position_long,timestamp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,120,0,48.52734,9.199440,953220649
2,0,120,0,48.52733,9.199429,953220650
3,1091,116,2137,48.52736,9.199243,953220658
4,2522,109,3387,48.52736,9.199049,953220662
5,2925,103,3798,48.52736,9.198994,953220663
6,4479,99,3919,48.52735,9.198785,953220667


Warning message in if (mins > 60) {:
“Bedingung hat Länge > 1 und nur das erste Element wird benutzt”


distance,hr,speed,pace,pos_lat,pos_long,time
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
0.000,120,0.0,NaN : NaN,48.52734,9.199440,0 : 0 : 0
0.000,120,0.0,NaN : NaN,48.52733,9.199429,0 : 0 : 1
0.011,116,7.7,NaN : 47,48.52736,9.199243,0 : 0 : 9
0.025,109,12.2,NaN : 55,48.52736,9.199049,0 : 0 : 13
0.029,103,13.7,NaN : 22,48.52736,9.198994,0 : 0 : 14
0.045,99,14.1,NaN : 15,48.52735,9.198785,0 : 0 : 18
0.049,104,13.9,NaN : 18,48.52735,9.198733,0 : 0 : 19
0.052,108,13.9,NaN : 18,48.52735,9.198684,0 : 0 : 20
0.056,115,13.9,NaN : 18,48.52735,9.198634,0 : 0 : 21


In [9]:
time <- 40789

hours <- time%/%3600
hours
mins <- time%%3600%/%60
mins
time%%60

[1] 11

[1] 19

[1] 49

# This part was used to analyze the data decoded by the lib

In [11]:
#rad
data <- read.fit('ACTIVITY/A8FF5035.FIT')
#wandern
data2 <- read.fit('ACTIVITY/A8691324.FIT')
#joggen
data3 <- read.fit('ACTIVITY/A6BG4226.FIT')
#schwimmen
data4 <- read.fit('ACTIVITY/A7CG0124.FIT')

In [11]:
data$session
data2$session
data3$session
data4$session

avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_heart_rate,max_speed,message_index,nec_lat,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
129,4354,4294967295,9,1,0,152,15060,0,48.52735,⋯,621,740,4294967295,588,3161446,8054058,7261254,24,4294967295,0


avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_cadence,max_fractional_cadence,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
56,34,125,650,4294967295,9,1,0,79,64,⋯,907,1481,8670,879,1144476,28127288,17603601,26,4294967295,0


avg_cadence,avg_fractional_cadence,avg_heart_rate,avg_speed,avg_stroke_count,event,event_type,first_lap_index,max_cadence,max_fractional_cadence,⋯,total_ascent,total_calories,total_cycles,total_descent,total_distance,total_elapsed_time,total_timer_time,total_training_effect,total_work,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
80,78,160,2589,4294967295,9,1,0,86,0,⋯,113,357,2370,124,460498,1778749,1778749,38,4294967295,0


avg_cadence,avg_speed,avg_stroke_distance,event,event_type,first_lap_index,max_speed,message_index,num_active_lengths,num_laps,⋯,sport,start_time,sub_sport,timestamp,total_calories,total_cycles,total_distance,total_elapsed_time,total_timer_time,trigger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
22,852,237,9,1,0,4124,0,36,12,⋯,5,963496884,17,963499487,383,760,180000,2602139,2596952,0


In [12]:
10**4/(852*6)

[1] 1.956182

In [ ]:
head(data$lap)
head(data2$lap)
head(data3$lap)
data4$lap

In [ ]:
head(data$record)
head(data2$record)
head(data3$record)
head(data4$record)

In [ ]:
#attribute units
attr(data4$session, 'units')

In [ ]:
to_kmh(4354/1000)
to_kmh(15060/1122)

to_duration(data4$session$total_elapsed_time)
data4$session$total_distance / 10**5


to_date(data4$session$timestamp)

In [29]:
data3$record$altitude

[1]   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA
 [16]   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA 4352 4340 4339 4341
 [31] 4343 4341 4343 4347 4350 4351 4357 4356 4355 4340 4318 4298 4286 4279 4269
 [46] 4254 4254 4252 4252 4253 4254 4254 4259 4261 4266 4271 4278 4290 4291 4297
 [61] 4303 4303 4304 4302 4302 4301 4303 4306 4304 4305 4301 4299 4302 4305 4305
 [76] 4310 4312 4314 4319 4326 4331 4338 4340 4347 4352 4353 4359 4360 4358 4362
 [91] 4366 4362 4361 4353 4353 4346 4342 4344 4353 4363 4368 4376 4384 4391 4400
[106] 4404 4404 4406 4406 4406 4408 4408 4411 4411 4412 4417 4415 4416 4417 4411
[121] 4404 4407 4407 4406 4404 4404 4404 4400 4398 4391 4388 4391 4390 4397 4400
[136] 4400 4399 4395 4403 4401 4401 4408 4422 4427 4431 4431 4431 4432 4430 4428
[151] 4427 4427 4423 4424 4423 4422 4414 4412 4414 4413 4410 4407 4396 4396 4398
[166] 4398 4396 4395 4395 4393 4390 4384 4381 4374 4376 4375 4374 4373 4377 4371
[181] 4368 4356 4349 4353 4341 4338 4333 4341 4350 4358 4360 4361 4358 4351 4347
[196] 4341 4334 4334 4336 4335 4331 4319 4314 4294 4285 4276 4281 4297 4307 4311
[211] 4319 4314 4318 4322 4323 4320 4323 4325 4335 4334 4336 4339 4336 4329 4323
[226] 4318 4317 4298 4295 4293 4291 4292 4293 4289 4285 4283 4281 4277 4262 4247
[241] 4237 4225 4220 4216 4220 4221 4230 4251 4270 4277 4275 4258 4249 4235 4225
[256] 4220 4218 4215 4213 4213 4212 4208 4201 4198 4199 4200 4202 4183 4207 4197
[271] 4216 4217 4225 4227 4231 4233 4236 4240 4242 4243 4244 4249 4262 4265 4275
[286] 4276 4278 4295 4301 4315 4329 4333 4334 4335 4334 4332 4332 4332 4337 4337
[301] 4336 4333 4340 4336 4343 4347 4353 4355 4359 4354 4350 4348 4345 4344 4343
[316] 4345 4346 4349 4350 4351 4352 4352 4353 4354 4364 4375 4378 4378 4371 4351
[331] 4348 4342 4339 4338 4338 4330 4329 4312 4309 4303 4298 4286 4287 4287 4284
[346] 4286 4281 4282 4283 4287